# Modelado de lenguaje a nivel de carácter en TensorFlow

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
## Reading and processing text

with open('/content/sample_data/1268-0.txt', 'r', encoding = 'utf8') as fp:
    text = fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
text = text[start_indx:end_indx]
char_set = set(text)
print('Total Length:', len(text))    

In [ ]:
print('Unique Characters:', len(char_set))

In [ ]:
## Building the dictionary to map 
## characters to integers, and reverse mapping

chars_sorted = sorted(char_set)

char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('Text encoded shape:', text_encoded.shape)
print(text[:15], '== Encoding ==>', text_encoded[:15])
print(text_encoded[15:21], '== Reverse ==>',
     ''.join(char_array[text_encoded[15:21]]))

In [ ]:
ds_text_encoded = tf.data.Dataset.from_tensor_slices(
    text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

In [ ]:
seq_length = 60
chunk_size = seq_length + 1
ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

## define the function for splitting x & y
def split_input_target(chunk):
    
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

In [ ]:
## Example of how ds_sequences is made up

for example in ds_sequences.take(2):
    print(' Input (x): ',
    repr(''.join(char_array[example[0].numpy()])))
    print('Target (y): ',
    repr(''.join(char_array[example[1].numpy()])))
    print()

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.LSTM(
    rnn_units,
    return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(vocab_size),
    ])
    return model

In [ ]:
## Setting the training parameters
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 1024

In [ ]:
tf.random.set_seed(1)
model = build_model(
    vocab_size=charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
model.summary()

In [ ]:
model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True
    ))

model.fit(ds, epochs=20)

In [ ]:
## Ejemplos

tf.random.set_seed(1)
logits = [[1.0, 1.0, 1.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

In [ ]:
tf.random.set_seed(1)
samples = tf.random.categorical(
logits=logits, num_samples=10)

tf.print(samples.numpy())

In [ ]:
def sample(model, starting_str,
    len_generated_text=500,
    max_input_length=60,
    scale_factor=2.0):
  
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
                 scaled_logits, num_samples=1)

        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()

        generated_str += str(char_array[new_char_indx])

        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
             [encoded_input, new_char_indx],
                            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]


    return generated_str

In [ ]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island'))